In [28]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from fredapi import Fred
from utils import crosscorr

## Отталкиваемся от GDP, так как у этого временного ряда самый широкий существующий диапазон значений 

In [46]:
# сюда вставить свой токен с https://fred.stlouisfed.org/docs/api/api_key.html
api_key = '2f65863001e3895e413ea46a51d481d7'
base_series = 'GDP'
fred = Fred(api_key = api_key)
data = fred.get_series(base_series)
data = pd.DataFrame({'date': data.index, base_series: data.values})
data = data.rename(columns = {"GDP": "GDP Nominal"})

## Далее приджойниваем остальные временные ряды

In [47]:
series_dict = {
    "GDPC1": "GDP Real",
    "USAPFCEQDSMEI": "Private Consumtion",
    "GPDI": "Investments", # не сходятся значения с тем, что в файле
    "PRFI": "Residential",
    "PNFI": "Non-Residential",
    "NINVRSAXDCUSQ": "Changes in inventories", # не сходятся значения с тем, что в файле
    "NETEXP": "Total export", # не сходятся значения с тем, что в файле
    "IMPGS": "Total import",
    "GCE": "Government Consumption Expenditures",
    "CONSDEF": "Consumption Deflator (PCE)",
    "CPIAUCSL": "CPI, Index", # не сходятся значения с тем, что в файле
#     "CPIAUCSL": "Industrial production Index",
    "UNRATE": "Unemployment", # не сходятся значения с тем, что в файле
    "PCE": "PCE" # не сходятся значения с тем, что в файле
}

for s in series_dict.keys():
    series_data = fred.get_series(s)
    series_data = pd.DataFrame({'date': series_data.index, series_dict[s]: series_data.values})
    data = data.merge(series_data, how = "left", on = "date")
    
    

In [48]:
data_needed_years = data[
    (data["date"]>="1970-01-01") & 
    (data["date"]<="2020-12-31")
]

In [49]:
data_needed_years.head(5)

,date,GDP Nominal,GDP Real,Private Consumtion,Investments,Residential,Non-Residential,Changes in inventories,Total export,Total import,Government Consumption Expenditures,Consumption Deflator (PCE),"CPI, Index",Unemployment,PCE
96,1970-01-01,1051.200,4939.759,1.579212e+11,168.113,42.493,123.796,NaN,3.514,53.517,247.889,17.416415,37.9,3.9,628.7
97,1970-04-01,1067.375,4946.770,1.603925e+11,171.455,41.365,125.008,NaN,5.207,55.204,249.143,17.629837,38.5,4.6,636.0
98,1970-07-01,1086.059,4992.357,1.633705e+11,173.904,42.566,126.255,NaN,4.088,56.431,254.584,17.806946,38.9,5.0,648.5
99,1970-10-01,1088.608,4938.857,1.650402e+11,166.754,47.218,123.520,NaN,2.985,57.888,258.708,18.027467,39.4,5.5,658.3
100,1971-01-01,1135.156,5072.996,1.697965e+11,189.495,50.961,126.281,NaN,4.570,58.651,261.905,18.182825,39.9,5.9,676.1


In [43]:
data_needed_years.to_csv("data/Macro_data.xlsx")